In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import isBtoDstlnu, whichBisSig, customMCmatching, B_ID

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-13 09:26:13.337695


In [3]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [4]:
sub = "sub02"

In [109]:
save_data = True
save_preprocessedDataframe = True
tmp_data = True
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [110]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "TMP_Dstlnu_SHR_BsX_Ablations/"
root_subdir = "SHR_CMvars_run1/"   # "SHR_LeasVars_run2/"

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir # + sub + "/"

In [7]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [8]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")
#afsPath = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/"
#fileY4S = uproot.open(afsPath + "DXtagDstl.root:variables")

In [9]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    #filename = afsPath + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_muons.root:variables


KeyInFileError: not found: 'variables' (with any cycle number)

    Available keys: (none!)

in file /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_tmp_muons.root

In [15]:
df_FSPs = pd.concat(dfs)

In [16]:
df_Y4S = fileY4S.arrays(library="pd")

In [18]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

6889895
159421


In [19]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [20]:
df_Y4S = df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [21]:
print(df_FSPs.shape[0])

6888332


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [22]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [23]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

df_FSPs.shape[0]: 6888332
groupsFSPs_uniqParID.shape[0]: 5037569
df_Y4S.shape[0]: 159421


In [24]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

df_FSPs[basf2_used].value_counts(): 0.0    3760206
1.0    3128126
Name: basf2_used, dtype: int64
df_FSPs[basf2_used].value_counts(): 1.0    3126242
0.0    1911327
Name: basf2_used, dtype: int64


In [25]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

df_FSPs.shape[0]: 5037569


## take a sample if used in notebook for faster processing

In [26]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

159207

In [27]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

100000

In [28]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [29]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

df_FSPs.shape[0]: 5037569
numEvents: 159207


## save sample 

In [30]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [31]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

## filter the wanted D*lnu and Hc isSignal==1 events

In [32]:
df_Y4S['isBtoDstlnu'] = df_Y4S.apply(isBtoDstlnu, axis=1)

In [33]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

97900

In [34]:
df_Y4S = df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

In [35]:
# delete FSPs for which no Y4S file entry is left after filer
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]
df_FSPs.shape[0]

2937886

### add cols with extra info for data prod

In [36]:
df_Y4S['Bsig_uniqParID'] = df_Y4S.apply(whichBisSig, axis=1)

In [37]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

### print one event

In [47]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)][["__event__",'Bsig_uniqParID','isBtoDstlnu','Hc_isSignalAcceptMissingGamma','Hc_mcPDG','Hc_genMotherPDG','Hc_uniqParID','Hc_genMotherID']][:10]

,__event__,Bsig_uniqParID,isBtoDstlnu,Hc_isSignalAcceptMissingGamma,Hc_mcPDG,Hc_genMotherPDG,Hc_uniqParID,Hc_genMotherID
0,22239789,83886081.0,1,1.0,421.0,413.0,100663296.0,3.0
1,22242808,83886082.0,1,1.0,421.0,413.0,100663296.0,3.0
2,22243407,83886081.0,1,1.0,-411.0,-413.0,100663296.0,6.0
3,22246415,83886081.0,1,1.0,-421.0,-423.0,100663296.0,13.0
4,14790197,83886082.0,1,1.0,-411.0,511.0,100663296.0,1.0
5,14794866,83886082.0,1,1.0,-421.0,-413.0,100663296.0,3.0
6,26364865,83886082.0,1,1.0,-431.0,-511.0,100663296.0,1.0
7,26367433,83886082.0,1,1.0,-411.0,-413.0,100663296.0,3.0
8,26369939,83886081.0,1,1.0,411.0,-511.0,100663296.0,2.0
9,19881295,83886082.0,1,1.0,-4122.0,-4222.0,100663296.0,4.0


In [26]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

-413.0       11839
 413.0       11706
-511.0       10713
 511.0       10670
-423.0        1813
 423.0        1794
 433.0         859
-433.0         843
 10431.0       338
-10431.0       330
-415.0         210
 415.0         196
-10413.0       107
 10413.0        84
 30443.0        65
 10411.0        61
-4212.0         61
 4222.0         60
 4212.0         59
 100443.0       56
-10411.0        56
 4112.0         49
-4114.0         48
-4222.0         48
 20433.0        42
 4114.0         42
-20433.0        41
-4112.0         37
 4214.0         32
-4214.0         30
 20443.0        28
 4224.0         25
-4224.0         24
 20413.0        14
-20413.0        13
 445.0           5
Name: Hc_genMotherPDG, dtype: int64

In [50]:
df_FSPs[df_FSPs["__event__"] == 22246415].sort_values('B_ID',ascending=False)[['mcPDG',
'B_ID',"basf2_used","basf2_Bsig","basf2_X",'genMothPDG_0', 'mcMother0_uniqParID', 'genMotherID_0',
'genMothPDG_1', 'mcMother1_uniqParID', 'genMotherID_1',
'genMothPDG_2', 'mcMother2_uniqParID', 'genMotherID_2',
'genMothPDG_3', 'mcMother3_uniqParID', 'genMotherID_3',
'genMothPDG_4', 'mcMother4_uniqParID', 'genMotherID_4',
'genMothPDG_5', 'mcMother5_uniqParID', 'genMotherID_5',
'genMothPDG_6', 'mcMother6_uniqParID', 'genMotherID_6',
'genMothPDG_7', 'mcMother7_uniqParID', 'genMotherID_7',
'genMothPDG_8', 'mcMother8_uniqParID', 'genMotherID_8',
'genMothPDG_9', 'mcMother9_uniqParID', 'genMotherID_9']]

,mcPDG,B_ID,basf2_used,basf2_Bsig,basf2_X,genMothPDG_0,mcMother0_uniqParID,genMotherID_0,genMothPDG_1,mcMother1_uniqParID,...,genMotherID_6,genMothPDG_7,mcMother7_uniqParID,genMotherID_7,genMothPDG_8,mcMother8_uniqParID,genMotherID_8,genMothPDG_9,mcMother9_uniqParID,genMotherID_9
245465,22.0,83886082,1.0,0.0,1.0,111.0,83886096.0,16.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90510,321.0,83886082,1.0,0.0,0.0,-421.0,83886102.0,22.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
286686,22.0,83886082,0.0,0.0,0.0,211.0,83886095.0,15.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
245464,22.0,83886082,1.0,0.0,1.0,111.0,83886096.0,16.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
286687,22.0,83886082,0.0,0.0,0.0,111.0,83886103.0,23.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90509,211.0,83886082,1.0,0.0,1.0,213.0,83886087.0,7.0,511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1140,-211.0,83886082,1.0,0.0,0.0,-421.0,83886102.0,22.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1139,-211.0,83886082,1.0,0.0,1.0,-415.0,83886086.0,6.0,511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90511,-321.0,83886081,1.0,1.0,0.0,421.0,83886088.0,8.0,413.0,83886083.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
112314,13.0,83886081,1.0,1.0,0.0,-511.0,83886081.0,1.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0


## check if category combinations make sense

In [40]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,1088901
1,0.0,1.0,0.0,11
2,1.0,0.0,0.0,343911
3,1.0,0.0,1.0,876155
4,1.0,1.0,0.0,628908


## delete the H_c FSPs (basf2_used==1 & basf2_Bsig==0 & basf2_X==0)

In [41]:
df_FSPs = df_FSPs[~((df_FSPs["basf2_used"]== 1) & (df_FSPs["basf2_Bsig"]== 0) & (df_FSPs["basf2_X"]== 0))]

In [42]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,1088901
1,0.0,1.0,0.0,11
2,1.0,0.0,1.0,876155
3,1.0,1.0,0.0,628908


## keep only the ones where Hc genMotherPDG is 511

In [48]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

 413.0       21987
-413.0       21976
-511.0       19952
 511.0       19781
-423.0        3404
 423.0        3371
 433.0        1623
-433.0        1613
-10431.0       655
 10431.0       639
 415.0         421
-415.0         397
-10413.0       220
 10413.0       182
 100443.0      124
 30443.0       118
 4212.0        118
-10411.0       115
 4222.0        105
 10411.0       105
-4222.0         98
 4112.0         94
-4212.0         90
 4114.0         83
-4114.0         82
-4112.0         81
 20433.0        76
-20433.0        69
 4214.0         61
 20443.0        60
-4214.0         59
 4224.0         43
-4224.0         41
-20413.0        30
 20413.0        21
 445.0           4
-10433.0         1
 10441.0         1
Name: Hc_genMotherPDG, dtype: int64

In [49]:
df_Y4S = df_Y4S[(df_Y4S['Hc_genMotherPDG'] == 511.0) | (df_Y4S['Hc_genMotherPDG'] == -511.0)]

df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [50]:
print(df_Y4S.shape[0])
print(df_FSPs.shape[0])

39733
1042500


## is B-sig really the B->D*lnu?

In [51]:
groupsBsig = pd.DataFrame({'count' : df_Y4S.groupby( ["Hc_motherUniqParID","Bsig_uniqParID"] ).size()}).reset_index()
groupsBsig

,Hc_motherUniqParID,Bsig_uniqParID,count
0,83886081.0,83886081.0,339
1,83886081.0,83886082.0,19475
2,83886082.0,83886081.0,19820
3,83886082.0,83886082.0,99


In [60]:
df_Y4S = df_Y4S[~(df_Y4S["Hc_motherUniqParID"] == df_Y4S["Bsig_uniqParID"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

## investigate the neutrals cut w/ goodBelleGamma

In [63]:
cuts = [0.050,0.100]

num_all_bg_neutrals = df_FSPs[((df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 

for cut in cuts:
    num_bg_cutNeutrals = df_FSPs[((df_FSPs["p"]<cut) & (df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 
    num_all_cutNeutrals = df_FSPs[((df_FSPs["p"]<cut) & (df_FSPs["PDG"]==22.))].shape[0]
    print("for cut p >",cut)
    print("this fraction of cut outs is bg:", round(num_bg_cutNeutrals/num_all_cutNeutrals,3))
    print("this fraction of bg is cut out:", round(num_bg_cutNeutrals/num_all_bg_neutrals,3))
    
num_bg_cutNeutrals = df_FSPs[((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 
num_all_cutNeutrals = df_FSPs[((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.))].shape[0]
print("for cut goodBelleGamma")
print("this fraction of cut outs is bg:", round(num_bg_cutNeutrals/num_all_cutNeutrals,3))
print("this fraction of bg is cut out:", round(num_bg_cutNeutrals/num_all_bg_neutrals,3))

for cut p > 0.05
this fraction of cut outs is bg: 0.908
this fraction of bg is cut out: 0.633
for cut p > 0.1
this fraction of cut outs is bg: 0.802
this fraction of bg is cut out: 0.857
for cut goodBelleGamma
this fraction of cut outs is bg: 0.874
this fraction of bg is cut out: 0.754


In [64]:
num_all_cutNeutrals # this is how many gamma will be cut with goodBelleGamma

355317

## cut on goodBelleGamma

In [69]:
df_FSPs.shape[0]

1033275

In [70]:
df_FSPs = df_FSPs[~((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.))]

In [71]:
df_FSPs.shape[0]

677958

## delete delete_top_numFSP_evts

In [72]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Y4S.shape[0]

In [75]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Y4S = df_Y4S[df_Y4S['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Y4S.shape[0]/df_Y4S.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 40
0.995
would delete 0.5 % of evts
new max FSPs: 28
0.996
would delete 0.5 % of evts
new max FSPs: 28
0.997
would delete 0.5 % of evts
new max FSPs: 28
0.998
would delete 0.26 % of evts
new max FSPs: 29
0.999
would delete 0.11 % of evts
new max FSPs: 31


In [76]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.998
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Y4S = df_Y4S[df_Y4S['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])
print("deleted", round(100 - (df_Y4S.shape[0]//numEvtsBeforeCut)*100,2),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())


677958
39295
used delete_percentileLvl: 0.998
count    39285.000000
mean        17.257426
std          4.118904
min          4.000000
25%         14.000000
50%         17.000000
75%         20.000000
max         40.000000
Name: count, dtype: float64
674688
39191
deleted 0.0026466471561267246 % of events
count    39285.000000
mean        17.257426
std          4.118904
min          4.000000
25%         14.000000
50%         17.000000
75%         20.000000
max         40.000000
Name: count, dtype: float64


## cut on momenta for neutrals, p>50MeV

In [29]:
#df_FSPs_final = df_FSPs_final[~((df_FSPs_final["p"]<0.050) & (df_FSPs_final["PDG"]==22.))]

In [30]:
#print(df_FSPs_final.shape[0])
#print(df_Y4S_final.shape[0])

1024354
53494


## impute the nan values of variables

In [100]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0


## save the dfs as they will be used for data prod

In [101]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Y4S

In [102]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/'

In [103]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_SHR_gamma50cut.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_SHR.csv")

# start of NN data creation

In [104]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 4
maxFSPs: 29 

num Events: 39182 



In [107]:
variable_combinations = [
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_allExtras"],
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID"],"SUPA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID"],"SUPA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID"],"SUPA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID"],"SUPA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ],"SUPA_no_muID"],
    [[],"SUPA_only4mom"]
]

In [111]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    #minFSPs = 5
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    

                basf2_usage = "basf2_NONE"
                if particle["basf2_Bsig"] == 1.0:
                    basf2_usage = "basf2_Bsig"
                elif particle["basf2_X"] == 1.0:
                    basf2_usage = "basf2_X"
                elif particle["basf2_used"] == 0:
                    basf2_usage = "basf2_bg"

                global_tag_Info = str((particle["mcPDG"])) 
                global_tag_Info += "_" + basf2_usage
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.06

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")
        #del df_num_subset


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/TMP_Dstlnu_SHR_BsX_Ablations/SHR_CMvars_run1/SUPA_allExtras is True 

numEvents: 1
num_FSPs_toData: 4
skipped because <10 events 

numEvents: 5
num_FSPs_toData: 5
skipped because <10 events 

numEvents: 36
num_FSPs_toData: 6
leaves.shape: (36, 6, 14)
SA_target.shape: (36, 6)
global_tag.shape: (36, 7)
leaves[0]: [[-2.82296270e-01  1.24597120e+00  2.46626481e-01  1.30542090e+00
   1.05658367e-01 -1.00000000e+00  1.65630006e-03 -3.69703906e-02
   2.00000000e+00  9.83398438e-01  3.73868797e-12  8.75681364e-19
   2.53875338e-56  1.00000000e+00]
 [-1.55037153e+00  9.40281689e-01  1.58184779e+00  2.41029124e+00
   1.39570385e-01  1.00000000e+00  1.07741070e-03 -3.78275517e-02
   2.00000000e+00  1.00000000e+00  8.17980131e-01  1.66051148e-06
   8.29847193e-10  1.82018208e-01]
 [ 1.55090792e-02 -1.17995381e-01 -7.34342262e-02  1.97575099e-01
   1.39570385e-01  1.00000000e+00  2.47451864e-02 -4.31642743e-02
  -1.


numEvents: 895
num_FSPs_toData: 10
leaves.shape: (895, 10, 14)
SA_target.shape: (895, 10)
global_tag.shape: (895, 11)
leaves[0]: [[-1.10866559e+00 -1.81333169e-01 -1.93679661e-01  1.13997069e+00
   5.10998943e-04  1.00000000e+00  2.34084699e-03 -7.79032713e-02
   2.00000000e+00  9.63867188e-01  2.48975647e-09  6.14821062e-32
   9.99999998e-01  2.09905260e-14]
 [-4.10492159e-02 -2.17558742e-01  5.57650864e-01  7.76986680e-01
   4.93676990e-01  1.00000000e+00  9.68043785e-03 -1.11022457e-02
   1.00000000e+00  9.91210938e-01  9.59890159e-01  9.98899544e-04
   1.47876606e-07  3.91107936e-02]
 [ 8.63785520e-02 -6.70418203e-01 -1.25562660e-02  6.76076535e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.72656250e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 3.61032449e-02 -1.43939614e-01 -6.18041307e-02  1.60753872e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  1.00000000e+00 -1.00


numEvents: 2423
num_FSPs_toData: 13
leaves.shape: (2423, 13, 14)
SA_target.shape: (2423, 13)
global_tag.shape: (2423, 14)
leaves[0]: [[ 6.85969815e-002  2.10905477e-001 -6.23105690e-002  2.30367691e-001
   0.00000000e+000  0.00000000e+000  1.18686603e-009  2.23517418e-010
   2.00000000e+000  9.79492188e-001 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [-1.53796569e-001 -4.22358699e-003  9.42677110e-002  1.80437315e-001
   0.00000000e+000  0.00000000e+000  1.18686603e-009  2.23517418e-010
   2.00000000e+000  9.38476562e-001 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [-8.65072906e-002  8.92775059e-002  2.61332780e-001  3.21292233e-001
   1.39570385e-001 -1.00000000e+000  2.05202872e-002 -1.80021419e-001
   1.00000000e+000  9.89257812e-001  3.61450815e-001  1.51473245e-011
   7.00729003e-006  6.38542178e-001]
 [ 1.68460906e+000 -1.25182927e+000  9.40599501e-001  2.29993736e+000
   5.10998943e-004  1.00000000e+000  5.76292728e-004  2


numEvents: 3626
num_FSPs_toData: 16
leaves.shape: (3626, 16, 14)
SA_target.shape: (3626, 16)
global_tag.shape: (3626, 17)
leaves[0]: [[ 3.63808542e-01  3.14406335e-01  4.18483526e-01  6.37445261e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.73632812e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-6.41749278e-02  1.10787183e-01  1.32663446e-02  5.10181524e-01
   4.93676990e-01 -1.00000000e+00  1.57903190e-01  9.53131744e-01
  -1.00000000e+00 -1.00000000e+00  9.83918743e-01  5.42292422e-07
   9.44582286e-20  1.60806336e-02]
 [ 6.48649409e-02  1.43321931e-01  5.98712899e-02  1.68324710e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-7.44853839e-02 -6.74118876e-01  1.20599950e-02  6.92538568e-01
   1.39570385e-01  1.00000000e+00  4.65625497e-03 -3.35469795e-03
   2.00000000e+00  9.81445312e-01  


numEvents: 3642
num_FSPs_toData: 18
leaves.shape: (3642, 18, 14)
SA_target.shape: (3642, 18)
global_tag.shape: (3642, 19)
leaves[0]: [[-2.28277333e-02  1.77047297e-01 -9.75657403e-02  2.03435308e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.82421875e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 2.29198158e-01  6.86485022e-02 -2.24435896e-01  3.28048600e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   3.00000000e+00  9.84375000e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-2.73139477e-02 -1.05862223e-01  1.25664935e-01  2.17311827e-01
   1.39570385e-01 -1.00000000e+00  4.67086273e-02  9.61187643e-02
  -1.00000000e+00 -1.00000000e+00  8.72846948e-01  1.10894659e-12
   1.13697522e-01  1.34555296e-02]
 [-3.32447961e-02  1.68007761e-01 -6.73128217e-02  1.84018587e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.94140625e-01 -

...


numEvents: 115
num_FSPs_toData: 7
leaves.shape: (115, 7, 4)
SA_target.shape: (115, 7)
global_tag.shape: (115, 8)
leaves[0]: [[ 0.59659249  0.28716227  0.64437169  0.93438725]
 [ 0.07329267  0.07181919  0.08168085  0.19152406]
 [-0.44062054  0.08912396 -0.12828775  0.67990018]
 [ 1.11513817 -0.82226366 -0.48774457  1.47265377]
 [ 0.1541893  -0.01559391 -0.18708698  0.24293837]
 [-1.14457798  0.9793973  -1.0343014   1.82730873]
 [ 0.03022073  0.02460623  0.04815845  0.06195155]]
SA_target[0]: [2. 2. 2. 2. 0. 1. 2.]
global_tag[0]: [b'13.0_basf2_Bsig' b'-211.0_basf2_Bsig' b'321.0_basf2_Bsig'
 b'-13.0_basf2_Bsig' b'nan_basf2_Bsig' b'22.0_basf2_X' b'22.0_basf2_X'
 b'evt37407464']

numEvents: 267
num_FSPs_toData: 8
leaves.shape: (267, 8, 4)
SA_target.shape: (267, 8)
global_tag.shape: (267, 9)
leaves[0]: [[-0.46784204 -0.47263652 -0.27414843  0.87243097]
 [-0.05502246 -0.11018925  0.0135485   0.18663493]
 [ 0.04806399 -0.36540744  0.65253472  0.74942265]
 [ 1.19226778 -1.77801943  1.40759468 


numEvents: 3267
num_FSPs_toData: 15
leaves.shape: (3267, 15, 4)
SA_target.shape: (3267, 15)
global_tag.shape: (3267, 16)
leaves[0]: [[ 1.82807013e-01  5.10216951e-01  3.69986027e-01  8.21185955e-01]
 [-4.03148472e-01 -3.87657136e-01 -6.48287609e-02  5.80077069e-01]
 [-2.23172590e-01  1.95526525e-01  2.98758060e-01  6.48852815e-01]
 [ 9.32218015e-01 -4.71192002e-01 -5.52421272e-01  1.18983254e+00]
 [ 3.42781901e-01  8.29302549e-01 -1.64302438e-01  1.03728222e+00]
 [ 8.19911137e-02 -2.46846434e-02 -3.64084244e-02  1.67741885e-01]
 [-1.86390066e+00  5.69154203e-01  1.49040830e+00  2.45571635e+00]
 [ 2.76721358e-01  3.04751188e-01 -4.26603369e-02  4.13845262e-01]
 [-5.76223843e-02  9.59186330e-02 -3.12036891e-02  1.16165372e-01]
 [-1.29782170e-01  1.64932630e-03  2.33697101e-01  2.67320907e-01]
 [ 2.04804167e-02 -5.89771532e-02  8.01272765e-02  1.01578209e-01]
 [ 5.81036471e-02  4.69068885e-02  1.33790374e-01  1.53219301e-01]
 [ 5.03372699e-02  9.69544575e-02  2.90345866e-02  1.13035458e-


numEvents: 2537
num_FSPs_toData: 21
leaves.shape: (2537, 21, 4)
SA_target.shape: (2537, 21)
global_tag.shape: (2537, 22)
leaves[0]: [[-2.56460011e-01  7.65865564e-01  5.76211572e-01  1.00190891e+00]
 [ 2.74792850e-01 -3.38022932e-02 -2.47723743e-01  3.96863517e-01]
 [-1.44195080e-01  1.35893058e-02 -7.71479085e-02  1.64099635e-01]
 [-1.94691792e-02 -4.67226841e-02  2.63214950e-02  5.70515490e-02]
 [ 4.98395205e-01  4.51595515e-01  2.65268460e-02  6.87400797e-01]
 [ 6.08388484e-01 -4.53627527e-01  1.03645481e-01  7.78547852e-01]
 [ 2.16967523e-01  1.91705246e-02  3.04516345e-01  6.19588243e-01]
 [-1.77348144e-02 -1.57399520e-01 -4.20188159e-02  5.20165055e-01]
 [ 3.30854803e-02 -7.11706728e-02  3.46112438e-02  8.57779218e-02]
 [-2.72481620e-01  4.79638875e-02  1.25130331e-02  2.76953685e-01]
 [-2.66365230e-01 -8.49556476e-02  2.19979752e-02  2.80449298e-01]
 [-4.30430025e-02  4.81817722e-02  3.41047458e-02  7.30569430e-02]
 [ 3.10528912e-02 -3.06814201e-02  6.36284202e-02  7.71635111e-


numEvents: 607
num_FSPs_toData: 25
leaves.shape: (607, 25, 4)
SA_target.shape: (607, 25)
global_tag.shape: (607, 26)
leaves[0]: [[ 5.98412395e-01  1.20533800e+00  4.86582220e-01  1.43487388e+00]
 [ 4.99430448e-02 -1.02381319e-01  1.36957601e-01  1.78139346e-01]
 [-3.25568885e-01  2.49670856e-02  2.64808983e-01  4.42969687e-01]
 [ 7.15277940e-02  6.46653632e-03 -1.84291303e-02  7.41463030e-02]
 [ 3.03231161e-02 -4.72570807e-02  3.15614641e-02  6.44115600e-02]
 [-8.98890123e-02  2.54692346e-01  2.85409629e-01  3.92946411e-01]
 [ 3.63365076e-02  2.40890752e-03  4.08747345e-02  5.47438448e-02]
 [ 2.63544060e-02 -3.66042294e-02  1.09959938e-01  1.18851219e-01]
 [ 1.50072426e-02 -4.60619554e-02  3.36305834e-02  5.89740384e-02]
 [ 1.65983483e-01  1.03170939e-01  2.29510859e-01  5.78434927e-01]
 [-1.13286100e-01 -1.87857434e-01  2.73248196e-01  3.50412232e-01]
 [-3.61129045e-02 -1.00261092e-01  1.12596244e-01  1.55030134e-01]
 [-2.77430624e-01  1.55313060e-01  3.39041591e-01  4.85302988e-01]



numEvents: 94
num_FSPs_toData: 29
leaves.shape: (94, 29, 4)
SA_target.shape: (94, 29)
global_tag.shape: (94, 30)
leaves[0]: [[ 0.04754735 -0.07102131 -0.03009991  0.09061337]
 [ 0.17849725  0.05062919  0.08340435  0.20342289]
 [-0.10273272 -0.06904533  0.08231436  0.14865034]
 [ 0.05317103 -0.05061597  0.03752448  0.16210217]
 [ 0.04556952 -0.01000252  0.11723085  0.12617331]
 [-0.01604716 -0.08706477 -0.04499651  0.09930997]
 [ 0.18568487  0.05895527  0.11422508  0.22583615]
 [-0.27981097 -0.29511228 -0.16275132  0.45973179]
 [-0.33744395  0.10539317  0.08553348  0.38957927]
 [-0.01215081  0.09180962  0.06166623  0.11126263]
 [ 0.0773765   0.01214711  0.00799401  0.07873106]
 [ 0.06585628 -0.1213923  -0.05525824  0.14875017]
 [ 0.03542737 -0.14567533 -0.02848464  0.15260333]
 [ 0.02854565 -0.19719297  0.07113268  0.21156507]
 [-0.01798869  0.10355753  0.09552171  0.19912821]
 [ 0.64826506 -0.05788203  0.29765624  0.72916188]
 [ 0.31276038  0.38626033 -0.00250005  0.51623855]
 [-0.060

In [112]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-10-13 12:17:19.276292


In [ ]:
#why only one FSP in so many events?

In [59]:
df_num_subset = df_FSPs_final.copy()
df_num_subset = df_num_subset[df_num_subset['numFSPs'] == 1]

In [51]:
df_FSPs_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'PDG', 'B_ID', 'numFSPs'],
      dtyp

In [52]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,591463
1,0.0,1.0,0.0,1
2,1.0,0.0,1.0,479138
3,1.0,1.0,0.0,349332


In [3]:
fileY4S = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/DXtagDstl.root:variables")
df_Y4S = fileY4S.arrays(library="pd")

In [4]:
pions = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/pions.root:variables")
df_pions = pions.arrays(library="pd")

NameError: name 'event_df' is not defined

In [114]:
df_Y4S_final["Hc_mcPDG"].value_counts()

 411.0     12191
-411.0     12132
 413.0      2921
-421.0      2902
 421.0      2883
-413.0      2802
-431.0      1098
 431.0      1065
 443.0       831
 4122.0      154
-4122.0      137
 423.0        39
-423.0        36
Name: Hc_mcPDG, dtype: int64

In [6]:
df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

11